In [6]:
import requests
import json

# Define admin API URLs for Alice and Bob
CENTRAL_PARTY_ADMIN_URL = "http://localhost:8001"
FL_CLIENT_ADMIN_URL = "http://localhost:8101"


def create_invitation(admin_url):
    """Create a new invitation."""
    response = requests.post(f"{admin_url}/connections/create-invitation")
    return response.json()


def receive_invitation(admin_url, invitation):
    """Receive an invitation."""
    response = requests.post(
        f"{admin_url}/connections/receive-invitation", json=invitation
    )
    return response.json()


def send_message(admin_url, connection_id, message):
    """Send a message to a connection."""
    response = requests.post(
        f"{admin_url}/connections/{connection_id}/send-message",
        json={"content": message},
    )
    return response.json()


def get_messages(admin_url, connection_id):
    """Get all messages for a connection."""
    response = requests.get(f"{admin_url}/connections/{connection_id}/messages")
    return response.json()


def get_connection(admin_url, connection_id):
    """Get the connection details."""
    response = requests.get(f"{admin_url}/connections/{connection_id}")
    return response.json()

In [7]:
# Step 1: Create an invitation from Central Party
print("Central Party: Creating invitation...")
central_party_invitation = create_invitation(CENTRAL_PARTY_ADMIN_URL)
print("Central Party's invitation:", json.dumps(central_party_invitation, indent=2))

Central Party: Creating invitation...
Central Party's invitation: {
  "connection_id": "dfb2803d-4198-40ef-a526-a23f516c6445",
  "invitation": {
    "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation",
    "@id": "a9c95165-5319-46be-9315-ca5d817429fe",
    "label": "Central Party Agent",
    "recipientKeys": [
      "2H8FJK6u6Rw5pt8ne39T1bEzLF6ewJkCd6KcXNhD5MSV"
    ],
    "serviceEndpoint": "http://central-party-agent:8000"
  },
  "invitation_url": "http://central-party-agent:8000?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYTljOTUxNjUtNTMxOS00NmJlLTkzMTUtY2E1ZDgxNzQyOWZlIiwgImxhYmVsIjogIkNlbnRyYWwgUGFydHkgQWdlbnQiLCAicmVjaXBpZW50S2V5cyI6IFsiMkg4RkpLNnU2Unc1cHQ4bmUzOVQxYkV6TEY2ZXdKa0NkNktjWE5oRDVNU1YiXSwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vY2VudHJhbC1wYXJ0eS1hZ2VudDo4MDAwIn0="
}


In [8]:
# Step 2: FL Client receives Central Party's invitation
print("FL Client: Receiving invitation...")
fl_client_connection = receive_invitation(
    FL_CLIENT_ADMIN_URL, central_party_invitation["invitation"]
)
print("FL Client's connection:", json.dumps(fl_client_connection, indent=2))

FL Client: Receiving invitation...
FL Client's connection: {
  "invitation_mode": "once",
  "rfc23_state": "request-sent",
  "their_role": "inviter",
  "accept": "auto",
  "created_at": "2024-07-04 09:18:16.660081Z",
  "connection_protocol": "connections/1.0",
  "state": "request",
  "routing_state": "none",
  "my_did": "LH1WW8E5tMhem3ncZFjrpH",
  "invitation_msg_id": "a9c95165-5319-46be-9315-ca5d817429fe",
  "connection_id": "7978c2e9-f571-4103-91e9-2ea96e07f0c5",
  "invitation_key": "2H8FJK6u6Rw5pt8ne39T1bEzLF6ewJkCd6KcXNhD5MSV",
  "updated_at": "2024-07-04 09:18:16.662127Z",
  "their_label": "Central Party Agent",
  "request_id": "e9a53bdb-13d8-45ea-a66d-29930eaa8931"
}


In [9]:
# Check the connection status
fl_client_connection_id = fl_client_connection["connection_id"]
fl_client_connection = get_connection(FL_CLIENT_ADMIN_URL, fl_client_connection_id)
print(
    "FL Client's connection status:",
    json.dumps(fl_client_connection["state"], indent=2),
)

central_party_connection_id = central_party_invitation["connection_id"]
central_party_connection = get_connection(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id
)
print(
    "Central Party's connection status:",
    json.dumps(central_party_connection["state"], indent=2),
)

FL Client's connection status: "request"
Central Party's connection status: "response"


In [10]:
# Step 3: Bob sends a message to Alice
print("FL Client: Sending message to Central Party...")
message_response = send_message(
    FL_CLIENT_ADMIN_URL, fl_client_connection_id, "Hello Alice, this is Bob!"
)
print("Message response:", json.dumps(message_response, indent=2))

FL Client: Sending message to Central Party...
Message response: {}
